In [1]:
import numpy as np

In [4]:
eps = 0.002 # bound on realtive error
delta = 0.01 # bound on the probability of getting bigger error then the relative error eps
stream_dim=100000 #This can be very big, the domain of the hash functions (number of how many different items we might have)

Colum numbers: $\doteq\left\lceil\frac{e}{\epsilon}\right\rceil$

In [5]:
# number of columns
col_num = int(np.ceil(np.exp(1)/eps))
print(col_num)

1360


In [6]:
# number of rows
row_num  = int(np.ceil(np.log(1/delta)))
print(row_num)

5


**Let us make weakly unviersal hash functions**

In [7]:
n=col_num
m = stream_dim
print(n,m)

1360 100000


In [8]:
from sympy import isprime

In [9]:
for it in np.arange(m,2*m):
  #print(it)
  if isprime(int(it)):
    p=it
    print(p)
    break


100003


In [10]:
print(n,m,p)

1360 100000 100003


$M=\{0,1,\ldots,m-1\}$, $m$ can be big, <br>
$N= \{0,1,\ldots,n-1\}$, $n$ is small </br>
Let $p$ be a prime number such that $n\leq m \leq p \leq 2m$.

Let $\mathcal{H}\doteq\{h_{a,b}:M\to N| 1\leq a \leq p-1, 0\leq b \leq p-1\}$. This is a weakly 2-universal family.

$h_{a,b}(x)=((ax+b \quad \textrm{mod } p) \quad \textrm{mod } n) \quad \forall x \in\mathbb{Z}_p$

In [11]:
class Make_hash_functions:
  def __init__(self, n,m,p):
    self.p = p
    self.n = n
    self.m = m
  def make_h_ab(self,a,b):
    p=self.p
    n=self.n

    def h_ab(x):
      tmp=a*x+b
      tmp =  tmp % p
      tmp = tmp % n
      return tmp
    return h_ab

In [12]:
print(n,m,p)
make_hash_functions_pn = Make_hash_functions(n,m,p)

1360 100000 100003


In [13]:
# Let us select randomly row_num hash functions
indices_a = np.random.uniform(1,p,size=row_num).astype(int)
indices_b = np.random.uniform(0,p,size=row_num).astype(int)
print(indices_a,indices_b)

[84988 97216 89103 42680 54307] [27527 59542 85269 91296 54576]


In [15]:
# create the hash functions with parameters a and b
hash_family = [];
for it,_ in enumerate(indices_a):
    h=make_hash_functions_pn.make_h_ab(indices_a[it],indices_b[it])
    hash_family.append(h)


In [16]:
H = len(hash_family)
print(H)

5


In [17]:
# h maps from m to n, n <=m
print(m,n)


100000 1360


In [ ]:
if 0:
  # Check weak universality:
  # Let 0 <=i,j <=n
  i = 6 
  j= 5
  count = 0
  for it in np.arange(H):
    if hash_family[it](i) == hash_family[it](j):
      count = count +1
  print(f'We should have {count/H} <= {1/m}')

The $CM(\epsilon,\delta)$ data structure is
represented with a 2D array, called Count matrix


In [18]:
print(H)
print(col_num)
print(row_num)

5
1360
5


### Create a random stream

In [19]:
A=np.zeros(stream_dim)
CM=np.zeros([row_num,col_num])
for iter in np.arange(50000):
  
  # In many real applications the items in the stream are not uniformly distributed. 
  # Here we sample the items of the stream randomly from a mixture of a uniform and exponential distribution
  select = np.random.uniform()
  if select > 0.2:
    index=np.floor(np.random.exponential(scale=30)).astype(int) % stream_dim
  else:
    index=np.floor(np.random.uniform(stream_dim)).astype(int)

  c=np.ceil(np.random.uniform(10))
  A[index]=A[index]+c

  for it in np.arange(row_num):
    act_col = hash_family[it](index)
    CM[it,act_col]=CM[it,act_col]+c

In [20]:
print(f' number of items in the stream: {np.sum(A)}')
print(f' the max of the counts: {np.max(A)}')
print(f' the min of the counts: {np.min(A)}')

 number of items in the stream: 300265.0
 the max of the counts: 8011.0
 the min of the counts: 0.0


In [21]:
error_limit=eps*np.sum(A)
print(f'Error limit: {error_limit}')

Error limit: 600.53


In [22]:
# Let us check the estimated and true value for an item
index = 20
true_val = A[index]
print(f'True value: {true_val}')

CM_values=[]
for it in np.arange(row_num):
  act_col = hash_family[it](index)
  CM_values.append(CM[it,act_col])
estimated_val=np.min(CM_values)
print(f'Estimated value: {estimated_val}')

True value: 4022.0
Estimated value: 4038.0


In [23]:
# The first 8 columns
CM[:,:8]

array([[  48.,   14.,   71.,   63.,  379.,   36.,   63.,   44.],
       [  35.,   45.,  668.,   55.,   43.,   29.,   25.,   40.],
       [  59.,   43.,  296.,   68.,   36.,   49.,   21., 1134.],
       [  75.,   46.,   81.,   44.,   42., 2427.,   23.,   15.],
       [  42.,   71.,   46.,   45.,   33.,   39.,   39.,   51.]])